In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
from edward.models import Normal
from edward.models import Beta
from edward.models import Bernoulli, MultivariateNormalTriL

import numpy as np
import tensorflow as tf
from getEmbeddings import getEmbeddings
import matplotlib.pyplot as plt
import scikitplot.plotters as skplt
import os.path

In [2]:
IN_DIM = 300
CLASS_NUM = 2
LEARN_RATE = 0.0001
TRAIN_STEP = 20000
tensorflow_tmp = "tmp_tensorflow"

In [3]:
N = 16608
D = IN_DIM

In [4]:
tf.reset_default_graph()
def data():
    # Get the training and testing data from getEmbeddings
    # Read the Doc2Vec data
    train_data = np.load('../Data/xtr.npy')
    eval_data = np.load('../Data/xte.npy')
    train_labels = np.load('../Data/ytr.npy')
    eval_labels = np.load('../Data/yte.npy')
    train_labels = train_labels.reshape((-1, 1)).astype(np.int32)

    return train_data, train_labels.flatten() 

In [5]:
'''def neural_network(X):
    h = tf.nn.relu(tf.matmul(X, W_0) + b_0)
    h = tf.nn.relu(tf.matmul(h, W_1) + b_1)
    h = tf.nn.relu(tf.matmul(h, W_2) + b_2)
    h = tf.nn.relu(tf.matmul(h, W_3) + b_3)
    return tf.reshape(h, [-1])

ed.set_seed(42)
'''
def neural_network(X):
    h = tf.tanh(tf.matmul(X, W_0) + b_0)
    h = tf.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.tanh(tf.matmul(h, W_2) + b_2)
    h = tf.tanh(tf.matmul(h, W_3) + b_3)
    return tf.reshape(h, [-1])

ed.set_seed(42)

In [6]:
with tf.name_scope("model"):
    W_0 = Normal(loc=tf.zeros([D, 300]), scale=tf.ones([D, 300]),
                 name="W_0")
    W_1 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_1")
    W_2 = Normal(loc=tf.zeros([300, 300]), scale=tf.ones([300, 300]), name="W_2")
    W_3 = Normal(loc=tf.zeros([300, 1]), scale=tf.ones([300, 1]), name="W_3")
    b_0 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_0")
    b_1 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_1")
    b_2 = Normal(loc=tf.zeros(300), scale=tf.ones(300), name="b_2")
    b_3 = Normal(loc=tf.ones(1), scale=tf.ones(1), name="b_3")  
    X = tf.placeholder(tf.float32, [N, D], name="X")
    #y = Normal(loc=neural_network(X), scale=0.1 * tf.ones(N), name="y")
    y = Bernoulli(logits=neural_network(X))

In [7]:
X_train, y_train = data()

In [8]:
print(X_train.shape)
print(y_train.shape)

(16608, 300)
(16608,)


In [9]:
# INFERENCE
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
      loc = tf.get_variable("loc", [D, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [D, 300]))
      qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
      loc = tf.get_variable("loc", [300, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))
      qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
      loc = tf.get_variable("loc", [300, 300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 300]))
      qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_3"):
      loc = tf.get_variable("loc", [300, 1])
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 1]))
      qW_3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
      loc = tf.get_variable("loc", [300])
      scale = tf.nn.softplus(tf.get_variable("scale", [300]))
      qb_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_3"):
      loc = tf.get_variable("loc", [1])
      scale = tf.nn.softplus(tf.get_variable("scale", [1]))
      qb_3 = Normal(loc=loc, scale=scale)

In [10]:
'''
zero_initializer = tf.constant_initializer(value=0.0)
ones_initializer = tf.constant_initializer(value=1.0)
with tf.variable_scope("posterior"):
    with tf.variable_scope("qW_0"):
      loc = tf.get_variable("loc", [D, 1000], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [D, 1000], initializer=ones_initializer))
      qW_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_1"):
      loc = tf.get_variable("loc", [1000, 500], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1000, 500], initializer=ones_initializer))
      qW_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_2"):
      loc = tf.get_variable("loc", [500, 300], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [500, 300], initializer=ones_initializer))
      qW_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qW_3"):
      loc = tf.get_variable("loc", [300, 1], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [300, 1], initializer=ones_initializer))
      qW_3 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_0"):
      loc = tf.get_variable("loc", [1000], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1000], initializer=ones_initializer))
      qb_0 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_1"):
      loc = tf.get_variable("loc", [500], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [500], initializer=ones_initializer))
      qb_1 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_2"):
      loc = tf.get_variable("loc", [300], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [300], initializer=ones_initializer))
      qb_2 = Normal(loc=loc, scale=scale)
    with tf.variable_scope("qb_3"):
      loc = tf.get_variable("loc", [1], initializer=zero_initializer)
      scale = tf.nn.softplus(tf.get_variable("scale", [1], initializer=ones_initializer))
      qb_3 = Normal(loc=loc, scale=scale)
'''

'\nzero_initializer = tf.constant_initializer(value=0.0)\nones_initializer = tf.constant_initializer(value=1.0)\nwith tf.variable_scope("posterior"):\n    with tf.variable_scope("qW_0"):\n      loc = tf.get_variable("loc", [D, 1000], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [D, 1000], initializer=ones_initializer))\n      qW_0 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_1"):\n      loc = tf.get_variable("loc", [1000, 500], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [1000, 500], initializer=ones_initializer))\n      qW_1 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_2"):\n      loc = tf.get_variable("loc", [500, 300], initializer=zero_initializer)\n      scale = tf.nn.softplus(tf.get_variable("scale", [500, 300], initializer=ones_initializer))\n      qW_2 = Normal(loc=loc, scale=scale)\n    with tf.variable_scope("qW_3"):\n      loc = tf.get_variable("loc", [300, 1],

In [11]:
#lets do inference
inference = ed.KLqp({W_0: qW_0, b_0: qb_0,
                   W_1: qW_1, b_1: qb_1,
                   W_2: qW_2, b_2: qb_2,
                   W_3: qW_3, b_3: qb_3}, data={X: X_train, y: y_train})
inference.run(logdir='log', n_iter=1000)

C:\Users\dcsbhr\AppData\Local\Continuum\anaconda3\envs\edward_env\lib\site-packages\edward\util\random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
C:\Users\dcsbhr\AppData\Local\Continuum\anaconda3\envs\edward_env\lib\site-packages\edward\util\random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


1000/1000 [100%] ██████████████████████████████ Elapsed: 165s | Loss: 19070.117


In [12]:
ed.evaluate('categorical_accuracy', data={y: y_train, X: X_train})

0.14757948